In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [2]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq
model = ChatGroq(
    api_key=GROQ_API_KEY,
    model="llama3-70b-8192",
    temperature=0.9,
)